## Custom Resource Management: Writing Context Managers
- Whenever you need custom setup/teardown logic, you can write your own Context Manager.
- A context manager ensures that teardown always runs, even if errors occur in the block.  
- Two approaches: implement `__enter__`/`__exit__` in a class or use the simpler generator-based decorator.  

In [ ]:
class MyContextManager:
    def __init__(self, timeout):
        self.timeout = timeout

    def __enter__(self):
        print("Setup complete")
        return "a simple value"

    def __exit__(self, exception_type, exception_value, traceback):
        print(f"Teardown")

        # Commenting out since we replaced *args for explicit
        # exception_type, exception_value, traceback parameters
        
        # for arg in args:
        #     print(arg)

        return False

with MyContextManager(timeout=30) as cm:
    print(cm)
    print("Inside the block")
    raise ValueError("Simulated problem")

Setup complete
a simple value
Inside the block
Teardown


ValueError: Simulated problem

## The `@contextlib.contextmanager` Decorator
- Provided by the `contextlib` module to turn a generator into a context manager.  
- Decorated function needs exactly one `yield`.  
- Code before `yield` runs as `__enter__`; code after (or in `finally`) runs as `__exit__`.  
- Simplifies many common patterns without writing a full class.

###  Generator Structure for `@contextmanager`
- Wrap the `yield` in `try...finally` to ensure teardown even on errors.  
- The value yielded is bound to `as var` in the `with` statement (if used).  
- You can catch exceptions inside the generator if you want to suppress them.

In [2]:
import os
from contextlib import contextmanager

@contextmanager
def change_directory(destination):
    """
    Temporarily switch into destination. If the directory does not exist,
    it is created just before the switch.

    Args:
        destination (str): Path to the directory that should become the working directory
    """

    origin_dir = os.getcwd()

    try:
        print(f"Changing into {destination}")
        os.makedirs(destination, exist_ok=True)
        os.chdir(destination)
        yield os.getcwd()
    finally:
        print(f"Reverting to original dir: {origin_dir}")
        os.chdir(origin_dir)

print(f"Start: {os.getcwd()}")

with change_directory("temp_dir") as new_dir:
    print(f"Inside: {new_dir}")

print(f"End: {os.getcwd()}")

Start: c:\Users\rhsro\OneDrive\Desktop\Python for Devops\python-devops\error-handling
Changing into temp_dir
Inside: c:\Users\rhsro\OneDrive\Desktop\Python for Devops\python-devops\error-handling\temp_dir
Reverting to original dir: c:\Users\rhsro\OneDrive\Desktop\Python for Devops\python-devops\error-handling
End: c:\Users\rhsro\OneDrive\Desktop\Python for Devops\python-devops\error-handling
